In [2]:



comm_module = CommModuleSocketTcp("Sock#1") 
my_telegram = Telegram(">ii1s1s",'Telegram1001',"int1 int2 text1 text2")
my_telegram.data.int1 = 2
my_telegram.data.int2 = 3
my_telegram.data.text1 = b"s"
my_telegram.set_string_value("text2","A")

#my_telegram.data.pic = b"\x02\x02\x02\x02\x02\x02\x02\x02\x02\x09"
my_telegram.telegram_pack()

test_binary_data = b'\x00\x00\x00\x02\x00\x00\x00\x03Q\x09'
my_telegram.telegram_unpack(test_binary_data)
my_telegram.data[1]
#my_telegram.get_string_value("text2")

23:27:37: Communication module Sock#1: created
23:27:37: Telegram Telegram1001: no communication module set
23:27:37: Telegram Telegram1001: with format: >ii1s1s; and variables: int1 int2 text1 text2  - created


3

In [5]:
from mtpbox import *

#initialization of logging
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.DEBUG,datefmt="%H:%M:%S")

###################main control loop######################
#recieve data from configured partner
#analyze/process recieved
#send responce data

#if we see partners any other partner - close connection
partner_plc_address = "192.168.99.1"
partner_plc2_address = "192.168.99.2"

#create communication partner table: IP<->Partner IP+Port
partner_table = {}

#reset partner-port assignment
partner_table[partner_plc_address] = None

#Master program
if __name__ == "__main__":
    test_loop_number = 20
    
    #create server and telegrams
    comm_module = CommModuleSocketTcp("Sock#1") 
    comm_module.set_partners_filter([partner_plc_address])
    comm_module.start_communication()
    
    #prepare telegrams
    partner_plc_telegram_1 = Telegram(">i2xi2x1s2x16s2s",'Telegram1',"int1 int2 text1 text2 crlf")
    partner_plc_telegram_1.data.int1 = 25
    partner_plc_telegram_1.data.int2 = 26
    partner_plc_telegram_1.set_string_value("text1","!")
    partner_plc_telegram_1.set_string_value("text2",time.strftime("Time is %H:%M:%S"))
    partner_plc_telegram_1.set_string_value("crlf","\n\r")
    
    for loop in range(test_loop_number):
        
        #check if partner table has unnassigned members. It means this IP hasn't connected before.
        if any(map(lambda x: x==None, partner_table.values())):
            for k,v in partner_table.items():
                if not v:
                    #check if partner hasn't been connected yet
                    #try to find it in comm_partners
                    for p in comm_module.partners:
                        if re.search("^"+k, p):
                            partner_table[k] = p
                            logging.info("Master: Partner '%s' assigned to '%s'",k,p)
                            break
        
        #remove unused partners from communication
        for p in comm_module.partners:
            if p not in partner_table.values():
                logging.debug("Master: Partner '%s' not used in master program -> removing",p)
                comm_module.remove_partner(p)
                
        #check if partner disconnected -> deassign connection in partner_table
        for k,v in partner_table.items():
            if v:
                if v not in comm_module.partners:
                    partner_table[k] = None
                    logging.info("Master: Partner '%s' has disconnected -> deassigning '%s'",k,v)        

        #####################################           
        ######   Main master logic  #########
        #####################################
        
        #some logic with partners
        #run through the partners list
        for k,v in partner_table.items():
            
            #check if is active. If not active we do nothing, but it depends of plan
            if v:
                
                #recive new messages
                data_rcv = comm_module.recieve_data(v)
                if data_rcv:
                    #some logic with recived data
                    logging.debug("Master: Data recived from %s:%s",k,data_rcv)
                
                #update partners telegram #TODO: useful to create Partner class
                partner_plc_telegram_1.data.int1 = 25
                partner_plc_telegram_1.data.int2 = 26
                partner_plc_telegram_1.set_string_value("text2",time.strftime("Tim1 is %H:%M:%S"))
                result = comm_module.send_data(partner_plc_telegram_1.telegram_pack(),v)
                
                partner_plc_telegram_1.set_string_value("text2",time.strftime("Tim2 is %H:%M:%S"))
                result = comm_module.send_data(partner_plc_telegram_1.telegram_pack(),v)
                #send messages to partners:
                
                
        #####################################           
        ######   Main master logic  #########
        #####################################

            
        time.sleep(1)
     
    #close communication, stop server
    comm_module.stop_communication()


23:03:20: Communication module Sock#1: created
23:03:20: Communication module Sock#1: set_partners_filter: new partner filter set: ['192.168.99.1']
23:03:20: Communication module Sock#1: start_communication: try to start communication treading
23:03:20: Communication module Sock#1: trying to start server
23:03:20: Communication module Sock#1: start_communication: communication threading started ok
23:03:20: Telegram Telegram1: no communication module set. NOT USED NOW
23:03:20: Telegram Telegram1: with format: >i2xi2x1s2x16s2s; and variables: int1 int2 text1 text2 crlf  - created
23:03:20: Communication module Sock#1: starting server...
23:03:20: Communication module Sock#1: server started
23:03:23: Communication module Sock#1: incoming partner name 192.168.99.1.57400
23:03:23: Communication module Sock#1: partner name 192.168.99.1.57400 accepted
23:03:23: Master: Partner '192.168.99.1' assigned to '192.168.99.1.57400'
23:03:23: Communication module Sock#1: recieve_data: input buffer 1

23:03:32: Communication module Sock#1: connection 192.168.99.1.57400: data sent
23:03:32: Communication module Sock#1: connection 192.168.99.1.57400: data sent
23:03:33: Communication module Sock#1: recieve_data: input buffer 192.168.99.1.57400 empty
23:03:33: Communication module Sock#1: send_data: data to '192.168.99.1.57400' added to main output buffer
23:03:33: Communication module Sock#1: send_data: data to '192.168.99.1.57400' added to main output buffer
23:03:33: Communication module Sock#1: checking main output buffer
23:03:33: Communication module Sock#1: found data for 192.168.99.1.57400
23:03:33: Communication module Sock#1: checking main output buffer
23:03:33: Communication module Sock#1: found data for 192.168.99.1.57400
23:03:33: Communication module Sock#1: connection 192.168.99.1.57400: data sent
23:03:33: Communication module Sock#1: connection 192.168.99.1.57400: data sent
23:03:34: Communication module Sock#1: recieve_data: input buffer 192.168.99.1.57400 empty
23:0

In [14]:
#comm_module.remove_partner("192.168.99.1.56751")
comm_module.remove_partner()

20:53:54: Communication module Sock#1: remove_partner: partner '192.168.99.1.56781' removed from communication list
20:53:54: Communication module Sock#1: remove_partner: partner '192.168.99.1.56782' removed from communication list
20:53:54: Communication module Sock#1: connection 192.168.99.1.56781 is not allowed anymore
20:53:54: Communication module Sock#1: connection 192.168.99.1.56782 is not allowed anymore
20:53:54: Communication module Sock#1: closing connection 192.168.99.1.56781
20:53:54: Communication module Sock#1: closing connection 192.168.99.1.56782


In [12]:
comm_module.get_partners()

['192.168.99.1.56778', '192.168.99.1.56779']

In [3]:
comm_module.stop_communication()

22:09:24: Communication module Sock#1: stop_communication: trying to stop threading
22:09:24: Communication module Sock#1: server stopped
22:09:24: Communication module Sock#1: stop_communication: threading stopped


In [8]:
from mtpbox import *

partner_plc_telegram_1 = Telegram(">i2xi2x1s2x16s",'Telegram1',"int1 int2 text1 text2")

partner_plc_telegram_1.telegram_pack()

b'\x00\x00\x00\x01\x00\x00\x00\x00\x00\x01\x00\x00A\x00\x00Time is 22:26:21'